<a href="https://colab.research.google.com/github/DeanImoke/Dean_DTSC3020_Fall2025/blob/main/Assignment_6_WebScraping_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 6 (4 points) — Web Scraping

In this assignment you will complete **two questions**. The **deadline is posted on Canvas**.


## Assignment Guide (Read Me First)

- This notebook provides an **Install Required Libraries** cell and a **Common Imports & Polite Headers** cell. Run them first.
- Each question includes a **skeleton**. The skeleton is **not** a solution; it is a lightweight scaffold you may reuse.
- Under each skeleton you will find a **“Write your answer here”** code cell. Implement your scraping, cleaning, and saving logic there.
- When your code is complete, run the **Runner** cell to print a Top‑15 preview and save the CSV.
- Expected outputs:
  - **Q1:** `data_q1.csv` + Top‑15 sorted by the specified numeric column.
  - **Q2:** `data_q2.csv` + Top‑15 sorted by `points`.


In [ ]:
1) #Install Required Libraries
!pip -q install requests beautifulsoup4 lxml pandas
print("Dependencies installed.")


### 2) Common Imports & Polite Headers

In [ ]:
# Common Imports & Polite Headers
import re, sys, pandas as pd, requests
from bs4 import BeautifulSoup
HEADERS = {"User-Agent": (
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/122.0 Safari/537.36")}
def fetch_html(url: str, timeout: int = 20) -> str:
    r = requests.get(url, headers=HEADERS, timeout=timeout)
    r.raise_for_status()
    return r.text
def flatten_headers(df: pd.DataFrame) -> pd.DataFrame:
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join([str(x) for x in tup if str(x)!="nan"]).strip()
                      for tup in df.columns.values]
    else:
        df.columns = [str(c).strip() for c in df.columns]
    return df
print("Common helpers loaded.")


## Question 1 — IBAN Country Codes (table)
**URL:** https://www.iban.com/country-codes  
**Extract at least:** `Country`, `Alpha-2`, `Alpha-3`, `Numeric` (≥4 cols; you may add more)  
**Clean:** trim spaces; `Alpha-2/Alpha-3` → **UPPERCASE**; `Numeric` → **int** (nullable OK)  
**Output:** write **`data_q1.csv`** and **print a Top-15** sorted by `Numeric` (desc, no charts)  
**Deliverables:** notebook + `data_q1.csv` + short `README.md` (URL, steps, 1 limitation)

**Tip:** You can use `pandas.read_html(html)` to read tables and then pick one with ≥3 columns.


In [ ]:
# --- Q1 Skeleton (fill the TODOs) ---
def q1_read_table(html: str) -> pd.DataFrame:
    """Return the first table with >= 3 columns from the HTML.
    TODO: implement with pd.read_html(html), pick a reasonable table, then flatten headers.
    """
    raise NotImplementedError("TODO: implement q1_read_table")

def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean columns: strip, UPPER Alpha-2/Alpha-3, cast Numeric to int (nullable), drop invalids.
    TODO: implement cleaning steps.
    """
    raise NotImplementedError("TODO: implement q1_clean")

def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort descending by Numeric and return Top-N.
    TODO: implement.
    """
    raise NotImplementedError("TODO: implement q1_sort_top")


In [ ]:
# Q1 — Write your answer here

import pandas as pd
import requests
def q1_read_table(html: str) -> pd.DataFrame:
  tables = pd.read_html(html)
  for t in tables:
    if t.shape[1] >= 3:
      df = t.copy()
      break
  df.columns = df.columns.str.strip()
  return df

def q1_clean(df: pd.DataFrame) -> pd.DataFrame:

  df.columns = [c.strip() for c in df.columns]

  col_map = {}
  for col in df.columns:
    name = col.lower()
    if "country" in name:
      col_map[col] = "Country"
    elif "alpha-2" in name:
      col_map[col] = "Alpha-2"
    elif "alpha-3" in name:
      col_map[col] = "Alpha-3"
    elif "numeric" in name:
      col_map[col] = "Numeric"
  df = df.rename(columns=col_map)

  cols = ["Country", "Alpha-2", "Alpha-3", "Numeric"]
  df = df[[c for c in cols if c in df.columns]]

  for c in ["Country", "Alpha-2", "Alpha-3"]:
    df[c] = df[c].astype(str).str.strip()
  df["Alpha-2"] = df["Alpha-2"].str.upper()
  df["Alpha-3"] = df["Alpha-3"].str.upper()

  df["Numeric"] = pd.to_numeric(df["Numeric"], errors="coerce").astype("Int64")
  df = df.dropna(subset = ["Country"])
  return df

def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
  df = df.sort_values(by="Numeric", ascending=False)
  return df.head(top)

if __name__ == "__main__":
  URL = "https://www.iban.com/country-codes"
  html = requests.get(URL).text
  df_raw = q1_read_table(html)
  df_clean = q1_clean(df_raw)
  df_clean.to_csv("data_q1.csv", index=False)
  top15 = q1_sort_top(df_clean, 15)
  print(top15)



                                               Country Alpha-2 Alpha-3  \
247                                             Zambia      ZM     ZMB   
246                                              Yemen      YE     YEM   
192                                              Samoa      WS     WSM   
244                                  Wallis and Futuna      WF     WLF   
240                 Venezuela (Bolivarian Republic of)      VE     VEN   
238                                         Uzbekistan      UZ     UZB   
237                                            Uruguay      UY     URY   
35                                        Burkina Faso      BF     BFA   
243                              Virgin Islands (U.S.)      VI     VIR   
236                     United States of America (the)      US     USA   
219                       Tanzania, United Republic of      TZ     TZA   
108                                        Isle of Man      IM     IMN   
113                                   

/tmp/ipython-input-3051605235.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)


## Question 2 — Hacker News (front page)
**URL:** https://news.ycombinator.com/  
**Extract at least:** `rank`, `title`, `link`, `points`, `comments` (user optional)  
**Clean:** cast `points`/`comments`/`rank` → **int** (non-digits → 0), fill missing text fields  
**Output:** write **`data_q2.csv`** and **print a Top-15** sorted by `points` (desc, no charts)  
**Tip:** Each story is a `.athing` row; details (points/comments/user) are in the next `<tr>` with `.subtext`.


In [ ]:
# --- Q2 Skeleton (fill the TODOs) ---
def q2_parse_items(html: str) -> pd.DataFrame:
    """Parse front page items into DataFrame columns:
       rank, title, link, points, comments, user (optional).
    TODO: implement with BeautifulSoup on '.athing' and its sibling '.subtext'.
    """
    raise NotImplementedError("TODO: implement q2_parse_items")

def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean numeric fields and fill missing values.
    TODO: cast points/comments/rank to int (non-digits -> 0). Fill text fields.
    """
    raise NotImplementedError("TODO: implement q2_clean")

def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort by points desc and return Top-N. TODO: implement."""
    raise NotImplementedError("TODO: implement q2_sort_top")


In [3]:
# Q2 — Write your answer here

import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

def q2_parse_items(html: str) -> pd.DataFrame:
    """Parse front page items into DataFrame columns:
       rank, title, link, points, comments, user (optional)."""

    soup = BeautifulSoup(html, 'lxml')
    items = []

    for row in soup.select('tr.athing'):
        item = {}


        rank_span = row.select_one('span.rank')
        item['rank'] = int(rank_span.get_text().replace('.', '').strip()) if rank_span else None


        titleline_span = row.select_one('span.titleline')
        if titleline_span:
            item['title'] = titleline_span.get_text().strip()
            link_tag = titleline_span.select_one('a')
            item['link'] = link_tag.get('href') if link_tag else None
        else:
            item['title'] = None
            item['link'] = None


        subtext_row = row.find_next_sibling('tr')
        if subtext_row:
            subtext_span = subtext_row.select_one('span.subtext')
            if subtext_span:

                score_span = subtext_span.select_one('span.score')
                item['points'] = score_span.get_text().replace(' points', '').strip() if score_span else None


                comments_link = subtext_span.find('a', text=re.compile(r'\d+\s+comments?'))
                if comments_link:
                    item['comments'] = comments_link.get_text().replace(' comments', '').replace(' comment', '').strip()
                elif subtext_span.find('a', text='discuss'):
                    item['comments'] = '0'
                else:
                    item['comments'] = None


                user_link = subtext_span.select_one('a.hnuser')
                item['user'] = user_link.get_text().strip() if user_link else None
            else:
                item['points'] = None
                item['comments'] = None
                item['user'] = None
        else:
            item['points'] = None
            item['comments'] = None
            item['user'] = None

        items.append(item)

    df = pd.DataFrame(items)
    return df


def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean numeric fields and fill missing values."""
    numeric_cols = ['rank', 'points', 'comments']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

    text_cols = ['title', 'link', 'user']
    for col in text_cols:
        if col in df.columns:
            df[col] = df[col].fillna('')

    return df


def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort by points desc and return Top-N."""
    return df.sort_values(by='points', ascending=False).head(top)



if __name__ == "__main__":
    URL = "https://news.ycombinator.com/"
    html_q2 = requests.get(URL).text

    df_q2 = q2_parse_items(html_q2)
    df_q2_cleaned = q2_clean(df_q2)
    df_q2_cleaned.to_csv('data_q2.csv', index=False)

    top_15_q2 = q2_sort_top(df_q2_cleaned)
    print("Top 15 rows sorted by points:")
    print(top_15_q2)


Top 15 rows sorted by points:
    rank                                              title  \
0      1         Why is Zig so cool? (nilostolte.github.io)   
1      2  Snapchat open-sources Valdi a cross-platform U...   
2      3  Mullvad: Shutting down our search proxy Leta (...   
3      4   Becoming a Compiler Engineer (rona.substack.com)   
4      5  Myna: Monospace typeface designed for symbol-h...   
5      6       How did I get here? (how-did-i-get-here.net)   
6      7  Immutable Software Deploys Using ZFS Jails on ...   
7      8             Ruby Solved My Problem (masilotti.com)   
8      9               Why I love OCaml (2023) (mccd.space)   
9     10  Cerebras Code now supports GLM 4.6 at 1000 tok...   
10    11  Can you save on LLM tokens using images instea...   
11    12  How to find your ideal customer, right away (r...   
12    13  YouTube Removes Windows 11 Bypass Tutorials, C...   
13    14                          FSF40 Hackathon (fsf.org)   
14    15  How a devboard 